In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import time, os, datetime
from selenium import webdriver
import sys
[sys.path.append(i) for i in ['.', '..']]
from crawling.utils.Database import Database

def del_html_tag(raw_text):
    return BeautifulSoup(raw_text, "lxml").text
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

In [2]:
db = Database()
db.connect()
# cursor = db.cursor()
query = """SELECT concat(IFNULL(a_sub1_name, ''), ' ', IFNULL(a_sub2_name, ''), ' ', IFNULL(a_sub3_name, '')) AS result FROM addr """
row = db.select_all(query)

# return to_df(row)
lines = pd.DataFrame({"data": [row[i]["result"] for i in range(len(row))]})
# lines = pd.read_sql(query, )
# lines.head()

In [3]:
lines["data"][0]

' 종로구 청운효자동'

In [4]:
# file_path = rf"{os.path.abspath('datas/pre_data/mango(1_226).txt')}"

# with open(file_path, 'rt', encoding='UTF8') as f:
#   lines = f.readlines()

# lines = [line.rstrip('\n') for line in lines]

diner_list = []
diner_url = []

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
browser = webdriver.Chrome(rf"{os.path.abspath('utils/chromedriver')}", chrome_options=chrome_options)

C:\Users\MERONA\AppData\Local\Temp\ipykernel_21084\2426900242.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(rf"{os.path.abspath('utils/chromedriver')}", chrome_options=chrome_options)
C:\Users\MERONA\AppData\Local\Temp\ipykernel_21084\2426900242.py:8: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(rf"{os.path.abspath('utils/chromedriver')}", chrome_options=chrome_options)


In [6]:
for data in lines["data"][18831:]:
  try:
    page_number = 1
    run=True
    last_page = 10
    print(data)
    while run:
      # 처음 접속해서 받아오는 정보, 상세정보에 들어가기 위한 url, pagination
      browser.get(f'https://www.mangoplate.com/search/{data}?keyword={data}&page={str(page_number)}')
      # time.sleep(1)
      
      html = browser.page_source
      soup = BeautifulSoup(html,'html.parser')
      figs = soup.find_all("figure", "restaurant-item")
      for i in range(len(figs)):
        # diner_list.append(figs[i].find("a")["href"])
        url = figs[i].find("a")["href"]
        diner_url.append([url, data])
      if(not figs or last_page == page_number):
        run = False
      else:
        page_number +=1

      # for i in diner_list:
        # if i not in diner_url:
  except:
    beepsound()
beepsound()

예천군  지내리
예천군  왕신리
예천군  청복리
예천군  통명리
예천군  우계리
예천군  갈구리
예천군  생천리
예천군  용산리
예천군  고평리
예천군  상동리
예천군  상금곡리
예천군  하금곡리
예천군  성현리
예천군  덕신리
예천군  방송리
예천군  노사리
예천군  구계리
예천군  죽림리
예천군  직리
예천군  두천리
예천군  사부리
예천군  내지리
예천군  원류리
예천군  선리
예천군  능천리
예천군  제곡리
예천군  대제리
예천군  하학리
예천군  포리
예천군  덕율리
예천군  관현리
예천군  천향리
예천군  진평리
예천군  벌방리
예천군  수한리
예천군  현내리
예천군  증거리
예천군  돈산리
예천군  유리
예천군  미석리
예천군  대맥리
예천군  장산리
예천군  마촌리
예천군  미호리
예천군  신월리
예천군  승본리
예천군  간방리
예천군  독양리
예천군  옥천리
예천군  기곡리
예천군  우래리
예천군  산성리
예천군  오신리
예천군  수계리
예천군  작곡리
예천군  오암리
예천군  본포리
예천군  원곡리
예천군  송곡리
예천군  금능리
예천군  한어리
예천군  본리
예천군  형호리
예천군  오천리
예천군  백송리
예천군  산합리
예천군  직산리
예천군  황지리
예천군  월포리
예천군  종산리
예천군  내신리
예천군  담암리
예천군  화지리
예천군  고산리
예천군  성평리
예천군  광전리
예천군  손기리
예천군  수심리
예천군  송전리
예천군  중평리
예천군  죽안리
예천군  마천리
예천군  화전리
예천군  사곡리
예천군  고림리
예천군  송지리
예천군  율현리
예천군  연천리
예천군  용암리
예천군  초적리
예천군  매산리
예천군  가리
예천군  읍부리
예천군  금남리
예천군  가야리
예천군  무이리
예천군  향석리
예천군  대은리
예천군  무지리
예천군  산택리
예천군  월오리
예천군  송암리
예천군  덕계리
예천군  금리
예천군  경진리
예천군  동송리
예천군  이사리
예천군  우감리
예천군  신음리
예천군  가곡리
예천군  황산리
예천군  장

In [7]:
diner_url

[['/restaurants/iycZL3nMVe', '예천군  우계리'],
 ['/restaurants/A61AhfAFkr', '예천군  우계리'],
 ['/restaurants/iycZL3nMVe', '예천군  우계리'],
 ['/restaurants/A61AhfAFkr', '예천군  우계리'],
 ['/restaurants/i0dizlGMM-', '예천군  하학리'],
 ['/restaurants/i0dizlGMM-', '예천군  하학리'],
 ['/restaurants/5BSl_NbzxJkL', '예천군  벌방리'],
 ['/restaurants/5BSl_NbzxJkL', '예천군  벌방리'],
 ['/restaurants/Cc6MHM8Fpw', '예천군  유리'],
 ['/restaurants/Cc6MHM8Fpw', '예천군  유리'],
 ['/restaurants/H2U-1etdsrAc', '예천군  작곡리'],
 ['/restaurants/TwDtMI3Cua', '예천군  작곡리'],
 ['/restaurants/wmuSXN4fAn', '예천군  작곡리'],
 ['/restaurants/H2U-1etdsrAc', '예천군  작곡리'],
 ['/restaurants/TwDtMI3Cua', '예천군  작곡리'],
 ['/restaurants/wmuSXN4fAn', '예천군  작곡리'],
 ['/restaurants/skGZ6N1K1TCV', '예천군  본리'],
 ['/restaurants/g9Vaw6v7lxPi', '예천군  본리'],
 ['/restaurants/vvNBF7RSscxH', '예천군  본리'],
 ['/restaurants/8hynfVgoG7', '예천군  본리'],
 ['/restaurants/zrIYKnVB9j71', '예천군  본리'],
 ['/restaurants/Z4wMkaVTt0hP', '예천군  본리'],
 ['/restaurants/Jvzsf23HPC', '예천군  본리'],
 ['/restaurants/-IUkVXWXH

In [8]:
diner_url

import csv
col = ["url", "addr"]
with open('url.csv', 'a', newline='') as f:
    write = csv.writer(f)
    write.writerow(col)
    write.writerows(diner_url)

In [4]:
diner_data = pd.read_csv(rf"{os.path.abspath('url.csv')}")
diner_url = diner_data["url"]
diner_url

0         /restaurants/MzWgvZ6d5vFs
1         /restaurants/hHN2RJInDQBX
2           /restaurants/XZ0HzSqJIF
3         /restaurants/9AM1_OLDQzt7
4           /restaurants/ZjlT6t7PRs
                    ...            
416077    /restaurants/b_uW1PVp6Kfu
416078      /restaurants/lKpVIRPEOz
416079    /restaurants/95xqRJN6jv34
416080    /restaurants/BU0SCnZ5GGaj
416081    /restaurants/UsND3V2pXF7n
Name: url, Length: 416082, dtype: object

In [5]:
diner_len = len(diner_url)

name_list = [0] * diner_len
rank_list = [0] * diner_len

review_list = [0] * diner_len
thumbnail_list = [0] * diner_len

addr_list = [[0]] * diner_len # 주소
call_list = [0] * diner_len # 전화번호
type_list = [0] * diner_len # 음식종류
pran_list = [0] * diner_len # 가격대
park_list = [0] * diner_len # 주차
time_list = [0] * diner_len # 영업시간
break_list = [0] * diner_len # 쉬는시간
holi_list = [0] * diner_len # 휴일
menu_list = [0] * diner_len # 메뉴
price_list = [0] * diner_len # 메뉴

In [42]:
# 테스트용 실행금지
# url = "https://www.mangoplate.com/restaurants/nWeCoq9KF7x6"
# # print(food_url[i])
# browser.get(url)
# html = browser.page_source
# soup = BeautifulSoup(html, 'html.parser')

# rank = soup.select_one('span > strong > span')
# print(del_html_tag(str(rank)))
# print(del_html_tag(str(rank)) if rank is not None else rank_list[20000])

None
0


In [6]:
for i in range(10000, 20000):
  url = "https://www.mangoplate.com" + diner_url[i]
  print(url)
  print(i)
  browser.get(url)
  time.sleep(2)
  html = browser.page_source
  soup = BeautifulSoup(html, 'html.parser')
  
  # 맛집의 상세정보(이름, 평점, 주소, 전화번호, 휴일, 메뉴, 이미지, 리뷰)
  # 이름
  title = soup.select_one('span > h1').get_text()
  sub_title = del_html_tag(str(soup.find("p", "branch")))
  if(len(sub_title) > 0):
    title += " " + sub_title
  # try:name_list.append(soup.select_one('span > h1').get_text())
  # except:name_list.append("")
  name_list[i] = title

  # 평점
  rank = soup.select_one('span > strong > span')
  # try:rank_list.append(soup.select_one('span > strong > span').get_text())
  # except:rank_list.append("")
  rank_list[i] = del_html_tag(str(rank)) if rank is not None else rank_list[i]

  # 정보
  tbody = soup.find('tbody')

  trs = tbody.find_all("tr")

  for j in range(len(trs)):
    info1 = trs[j].find("th", text="주소")
    if(info1 is not None):
      info1_tr = info1.parent
      addr = del_html_tag(str(info1_tr.find_all("td")[0])).split("지번")
      addrs = []
      for addr_ in addr:
        addrs.append(addr_.strip())
      addr_list[i] = addrs

    info2 = trs[j].find("th", text="전화번호")
    if(info2 is not None):
      info2_tr = info2.parent
      b = del_html_tag(str(info2_tr.find_all("td")))
      call_list[i] = b
    
    info3 = trs[j].find("th", text="음식 종류")
    if(info3 is not None):
      info3_tr = info3.parent
      c = del_html_tag(str(info3_tr.find_all("td")))
      type_list[i] = c[2:-2].split("/")
    
    info4 = trs[j].find("th", text="가격대")
    if(info4 is not None):
      info4_tr = info4.parent
      d = del_html_tag(str(info4_tr.find_all("td")))
      pran_list[i] = d
    
    info5 = trs[j].find("th", text="주차")
    if(info5 is not None):
      info5_tr = info5.parent
      e = del_html_tag(str(info5_tr.find_all("td")))
      park_list[i] = e
    
    info6 = trs[j].find("th", text="영업시간")
    if(info6 is not None):
      info6_tr = info6.parent
      f = del_html_tag(str(info6_tr.find_all("td")))
      time_list[i] = f
    
    info7 = trs[j].find("th", text="쉬는시간")
    if(info7 is not None):
      info7_tr = info7.parent
      g = del_html_tag(str(info7_tr.find_all("td")))
      break_list[i] = g
    
    info8 = trs[j].find("th", text="휴일")
    if(info8 is not None):
      info8_tr = info8.parent
      h = del_html_tag(str(info8_tr.find_all("td")))
      holi_list[i] = h
    
    info9 = trs[j].find("th", text="메뉴")
    if(info9 is not None):
      info9_tr = info9.parent
      j = info9_tr.find_all("td")[0]
      lis = j.find_all("li")
      menu = []
      price = []
      for li in lis:
        menu.append(li.find("span", "Restaurant_Menu").text)
        price.append(li.find("span", "Restaurant_MenuPrice").text)
      menu_list[i] = menu
      price_list[i] = price
  #리뷰
  review = soup.select_one('li:nth-child(1) > a > div.RestaurantReviewItem__ReviewContent > div > p')
  review_list[i] = review.get_text().strip() if review is not None else review_list[i]
  # try:review_list.append(review.get_text().strip())
  # except:review_list.append("")

  # 이미지
  img = ''
  try:
    img = soup.select_one('div:nth-child(1) > figure > figure > img')['src']
    # a = soup.select_one('div:nth-child(1) > figure > figure > img')['src']
  except TypeError:
    img = ''
  thumbnail_list[i] = img

  food_df = pd.DataFrame({'id':diner_url,'name':name_list,'rank':rank_list,'review':review_list,'thumbnail':thumbnail_list, 'addr':addr_list, 'call':call_list, 'type':type_list, 'price':pran_list, 'park':park_list, 'time':time_list, 'break':break_list, 'holiday':holi_list, 'menu':menu_list})
  

https://www.mangoplate.com/restaurants/sjbUF8D6Z38w
10000
https://www.mangoplate.com/restaurants/nzSVo_hEXMyK
10001
https://www.mangoplate.com/restaurants/-copMtLi2U
10002
https://www.mangoplate.com/restaurants/w9D_FFL-6-q7
10003
https://www.mangoplate.com/restaurants/1d79_7zsFzzz
10004
https://www.mangoplate.com/restaurants/SJvQhyV5V9lq
10005
https://www.mangoplate.com/restaurants/Ob72Bx8O6e1i
10006
https://www.mangoplate.com/restaurants/QC13D_U19E6p
10007
https://www.mangoplate.com/restaurants/kWUFCE-8_QbL
10008
https://www.mangoplate.com/restaurants/eXYh01FA-4dg
10009
https://www.mangoplate.com/restaurants/bbp-tTqgpIAM
10010
https://www.mangoplate.com/restaurants/ED0DzRAhU-Z1
10011
https://www.mangoplate.com/restaurants/2MNIy3CXyY
10012
https://www.mangoplate.com/restaurants/QG1r_P-yNUFi
10013
https://www.mangoplate.com/restaurants/CUsG8NAOSz
10014
https://www.mangoplate.com/restaurants/xaqxty6F_AMl
10015
https://www.mangoplate.com/restaurants/rsXdUpweaPbP
10016
https://www.mangopla

In [ ]:
food_df = pd.DataFrame({'id':diner_url,'name':name_list,'rank':rank_list,'review':review_list,'thumbnail':thumbnail_list, 'addr':addr_list, 'call':call_list, 'type':type_list, 'price':pran_list, 'park':park_list, 'time':time_list, 'break':break_list, 'holiday':holi_list, 'menu':menu_list})
# %%
print(food_df)
print("[MANGO_RESTAURANT] data to csv file")

dt = datetime.datetime.now()
fName = f'datas/restaurant_mango_{dt.year}_{dt.month}_{dt.day}.csv'
# fName = f'crawling/datas/restaurant_mango_{dt.year}_{dt.month}_{dt.day}.csv'
fName = rf'{os.path.abspath(fName)}'

food_df.to_csv(fName, sep=',', encoding='utf-8-sig', index=False)